In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score , classification_report , precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import chi2_contingency
import warnings
import os

In [ ]:
a1 = pd.read_excel("/content/case_study1.xlsx")
a2 = pd.read_excel("/content/case_study2.xlsx")

In [ ]:
print(a1.shape)
print(a2.shape)

(51336, 26)
(51336, 62)


In [ ]:
df1 = a1.copy()
df2 = a2.copy()

In [ ]:
df1.sample(10)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
48651,48652,7,1,6,1,1,0.143,0.143,0.857,0.143,...,1,2,0,0,1,1,6,3,11,4
33506,33507,1,0,1,0,0,0.000,0.000,1.000,0.000,...,0,0,0,0,0,0,1,1,58,58
13556,13557,5,3,2,0,0,0.000,0.000,0.400,0.600,...,0,0,0,0,1,4,1,1,77,12
29723,29724,10,3,7,7,3,0.700,0.300,0.700,0.300,...,0,5,1,0,0,1,9,4,14,3
42697,42698,10,8,2,0,1,0.000,0.100,0.200,0.800,...,0,3,0,1,1,3,7,3,56,7
17975,17976,1,0,1,0,0,0.000,0.000,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
31791,31792,1,0,1,1,0,1.000,0.000,1.000,0.000,...,0,0,0,0,0,1,0,1,3,3
25103,25104,1,1,0,0,1,0.000,1.000,0.000,1.000,...,0,1,0,0,0,0,1,0,13,13
33596,33597,2,0,2,0,0,0.000,0.000,1.000,0.000,...,0,0,0,1,0,2,0,1,49,16
14990,14991,3,2,1,0,0,0.000,0.000,0.333,0.667,...,0,0,1,0,0,3,0,0,100,9


In [ ]:
df1.info()
df1.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

,0
PROSPECTID,0
Total_TL,0
Tot_Closed_TL,0
Tot_Active_TL,0
Total_TL_opened_L6M,0
Tot_TL_closed_L6M,0
pct_tl_open_L6M,0
pct_tl_closed_L6M,0
pct_active_tl,0
pct_closed_tl,0


In [ ]:
# analyzed in excel to and saw that -99999 is null values
# if there are > 10000 null values we remove that column

column_to_remove_df1 = []
for i in df1.columns:
  if df1.loc[df1[i]==-99999].shape[0] >= 10000:
    column_to_remove_df1.append(i)

In [ ]:
column_to_remove_df1
# no columns null > 10000

[]

In [ ]:
# we need to remove those rows where any particular column has -99999
for i in df1.columns:
  df1 = df1.loc[df1[i] != -99999]

In [ ]:
df1.shape

# nearly 100 rows are removed

(51296, 26)

In [ ]:
# check for data 2
column_to_remove_df2 = []
for i in df2.columns:
  if df2.loc[df2[i] == -99999].shape[0] > 10000:
    column_to_remove_df2.append(i)

In [ ]:
column_to_remove_df2

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [ ]:
# these r the items that have > 10000 nulls
# so we need to reove them

df2 = df2.drop(column_to_remove_df2,axis=1)

In [ ]:
df2.shape

# columns are reduced from 62 to 54

(51336, 54)

In [ ]:
# now we remove the rows where the column have -99999

for i in df2.columns:
  df2 = df2.loc[df2[i] != -99999]

In [ ]:
df2.shape

#nearly 10000 row removed

(42066, 54)

In [ ]:
# now we need to merge these two data df1 and df2 , with a common column
# now finding the common column

for i in list(df1.columns):
  if i in list(df2.columns):
    print(i)

PROSPECTID


In [ ]:
# now we need to merge the two dataframe on PROSPECTID

df = pd.merge(df1,df2,how = 'inner',left_on=['PROSPECTID'],right_on = ['PROSPECTID'])

In [ ]:
df.shape

(42064, 79)

In [ ]:
df.sample(10)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
38536,47057,1,1,0,0,0,0.000,0.000,0.000,1.000,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
36462,44507,1,1,0,0,1,0.000,1.000,0.000,1.000,...,1.0,0.0,1.0,0.0,0,0,PL,ConsumerLoan,669,P2
30571,37314,13,6,7,3,1,0.231,0.077,0.538,0.462,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,AL,691,P2
23437,28634,1,0,1,0,0,0.000,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,668,P3
29377,35854,1,0,1,0,0,0.000,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,PL,PL,680,P2
19985,24424,5,2,3,0,1,0.000,0.200,0.600,0.400,...,0.0,0.0,0.0,0.0,0,0,others,PL,675,P2
35468,43309,5,4,1,0,0,0.000,0.000,0.200,0.800,...,0.0,0.0,0.0,0.0,0,0,others,others,700,P2
4998,6024,1,0,1,0,0,0.000,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,674,P2
6920,8336,2,1,1,0,0,0.000,0.000,0.500,0.500,...,0.0,0.0,0.0,0.0,0,0,others,others,671,P2
38520,47037,2,0,2,2,0,1.000,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,others,669,P2


In [ ]:
#now we need to check how many columns are categorical and numerical

categorical_column = []
for i in df.columns:
  if df[i].dtype == 'object':
    categorical_column.append(i)

In [ ]:
categorical_column.remove('Approved_Flag')

In [ ]:
categorical_column

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [ ]:
# numerical columns

numerical_column = []
for i in df.columns:
  if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
    numerical_column.append(i)

In [ ]:
numerical_column

['Total_TL',
 'Tot_Closed_TL',
 'Tot_Active_TL',
 'Total_TL_opened_L6M',
 'Tot_TL_closed_L6M',
 'pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'pct_active_tl',
 'pct_closed_tl',
 'Total_TL_opened_L12M',
 'Tot_TL_closed_L12M',
 'pct_tl_open_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'Auto_TL',
 'CC_TL',
 'Consumer_TL',
 'Gold_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'num_times_delinquent',
 'max_recent_level_of_deliq',
 'num_deliq_6mts',
 'num_deliq_12mts',
 'num_deliq_6_12mts',
 'num_times_30p_dpd',
 'num_times_60p_dpd',
 'num_std',
 'num_std_6mts',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_6mts',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_6mts',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'tot_enq',
 'CC_enq',
 'CC_enq_L6m',
 'CC_enq_L12m',
 'PL_enq',
 'PL_enq_L6m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L12m',
 'enq_L6m',
 'enq_L3m',

In [ ]:
print(len(numerical_column))
print(len(categorical_column))

72
5


In [ ]:
# now we need to do testing to see whether these particular column in related to target column
# our target column in categorical and it has 4 categories p1,p2,p3,p4

# for cateorical vs categorical --> we use chi2 square test
# for cateorical vs numerical , where categories <= 2  --> we use T_test
# for categorical vs numerical , where categories > 2 --> we use ANOVA test

# here there are 4 categories in target column
# Now doing chi2_square test

for i in categorical_column:
  chi2 , p_val ,_ , _ = chi2_contingency(pd.crosstab(df[i],df['Approved_Flag']))
  print(i , '---->  p_val :' , p_val)


# if we consider alpha , significance_value as 0.05 , ie we only consider 5% risk
# ho (Null Hypothesis): the variables are not associated with the target variable
# h1 (Alternative Hypothesis) : the variable is associated with the target variable
# if we have p_val <= alpha , so we reject null hypothesis
# therefore these variables are associated with the target


MARITALSTATUS ---->  p_val : 3.578180861038862e-233
EDUCATION ---->  p_val : 2.6942265249737532e-30
GENDER ---->  p_val : 1.907936100186563e-05
last_prod_enq2 ---->  p_val : 0.0
first_prod_enq2 ---->  p_val : 7.84997610555419e-287


In [ ]:
# for numerical cols we check VIF ( `variance inflation factor ) this identifies the multi-collinearity of the columns with each other

# we consider sequential VIF

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = df[numerical_column].copy()
columns_to_be_kept = []

# Work on a copy of the column list
current_columns = vif_data.columns.tolist()

i = 0
while i < len(current_columns):
    col_name = current_columns[i]

    vif_value = variance_inflation_factor(vif_data.values, i)
    print(f"Column '{col_name}' ---> VIF: {vif_value:.2f}")

    if vif_value <= 6:
        columns_to_be_kept.append(col_name)
        i += 1  # move to next column
    else:
        print(f"Removing column '{col_name}' due to high VIF")
        vif_data.drop(columns=[col_name], inplace=True)
        current_columns = vif_data.columns.tolist()  # update column list
        # don't increment i, since next column takes current index

print("\n Columns retained after VIF filtering:")
print(columns_to_be_kept)


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Column 'Total_TL' ---> VIF: inf
Removing column 'Total_TL' due to high VIF


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Column 'Tot_Closed_TL' ---> VIF: inf
Removing column 'Tot_Closed_TL' due to high VIF
Column 'Tot_Active_TL' ---> VIF: 11.32
Removing column 'Tot_Active_TL' due to high VIF
Column 'Total_TL_opened_L6M' ---> VIF: 8.36
Removing column 'Total_TL_opened_L6M' due to high VIF
Column 'Tot_TL_closed_L6M' ---> VIF: 6.52
Removing column 'Tot_TL_closed_L6M' due to high VIF
Column 'pct_tl_open_L6M' ---> VIF: 5.15
Column 'pct_tl_closed_L6M' ---> VIF: 2.61


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Column 'pct_active_tl' ---> VIF: inf
Removing column 'pct_active_tl' due to high VIF
Column 'pct_closed_tl' ---> VIF: 1788.79
Removing column 'pct_closed_tl' due to high VIF
Column 'Total_TL_opened_L12M' ---> VIF: 8.60
Removing column 'Total_TL_opened_L12M' due to high VIF
Column 'Tot_TL_closed_L12M' ---> VIF: 3.83
Column 'pct_tl_open_L12M' ---> VIF: 6.10
Removing column 'pct_tl_open_L12M' due to high VIF
Column 'pct_tl_closed_L12M' ---> VIF: 5.58
Column 'Tot_Missed_Pmnt' ---> VIF: 1.99


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Column 'Auto_TL' ---> VIF: inf
Removing column 'Auto_TL' due to high VIF
Column 'CC_TL' ---> VIF: 4.81
Column 'Consumer_TL' ---> VIF: 23.27
Removing column 'Consumer_TL' due to high VIF
Column 'Gold_TL' ---> VIF: 30.60
Removing column 'Gold_TL' due to high VIF
Column 'Home_TL' ---> VIF: 4.38
Column 'PL_TL' ---> VIF: 3.06
Column 'Secured_TL' ---> VIF: 2.90
Column 'Unsecured_TL' ---> VIF: 4.38
Column 'Other_TL' ---> VIF: 2.21
Column 'Age_Oldest_TL' ---> VIF: 4.92
Column 'Age_Newest_TL' ---> VIF: 5.21
Column 'time_since_recent_payment' ---> VIF: 3.39
Column 'num_times_delinquent' ---> VIF: 7.84
Removing column 'num_times_delinquent' due to high VIF
Column 'max_recent_level_of_deliq' ---> VIF: 5.26


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Column 'num_deliq_6mts' ---> VIF: inf
Removing column 'num_deliq_6mts' due to high VIF
Column 'num_deliq_12mts' ---> VIF: 7.38
Removing column 'num_deliq_12mts' due to high VIF
Column 'num_deliq_6_12mts' ---> VIF: 1.42
Column 'num_times_30p_dpd' ---> VIF: 8.08
Removing column 'num_times_30p_dpd' due to high VIF
Column 'num_times_60p_dpd' ---> VIF: 1.62
Column 'num_std' ---> VIF: 7.26
Removing column 'num_std' due to high VIF
Column 'num_std_6mts' ---> VIF: 15.60
Removing column 'num_std_6mts' due to high VIF
Column 'num_std_12mts' ---> VIF: 1.83
Column 'num_sub' ---> VIF: 1.51
Column 'num_sub_6mts' ---> VIF: 2.17
Column 'num_sub_12mts' ---> VIF: 2.62
Column 'num_dbt' ---> VIF: 2.30
Column 'num_dbt_6mts' ---> VIF: 7.36
Removing column 'num_dbt_6mts' due to high VIF
Column 'num_dbt_12mts' ---> VIF: 2.16
Column 'num_lss' ---> VIF: 2.87
Column 'num_lss_6mts' ---> VIF: 6.46
Removing column 'num_lss_6mts' due to high VIF
Column 'num_lss_12mts' ---> VIF: 2.85
Column 'recent_level_of_deliq' --

In [ ]:
len(columns_to_be_kept)

39

In [ ]:
# Anova test for columns to be kept

from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])
    b = list(df['Approved_Flag'])

    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)



In [ ]:
print(columns_to_be_kept_numerical)
print(len(columns_to_be_kept_numerical))
# 2 cols get dropped off

['pct_tl_open_L6M', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL', 'time_since_recent_payment', 'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd', 'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt', 'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'CC_enq_L12m', 'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever', 'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag']
37


In [ ]:
# listing all the final features
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]

In [ ]:
df.shape

(42064, 43)

In [ ]:
# Label encoding for the categorical features

['MARITALSTATUS', 'EDUCATION', 'GENDER' , 'last_prod_enq2' ,'first_prod_enq2']


['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [ ]:
print('Marital_status : ',df['MARITALSTATUS'].unique() )
print('Education :',df['EDUCATION'].unique())
print('Gender :',df['GENDER'].unique())
print('last_prod_enq2 :',df['last_prod_enq2'].unique())
print('first_prod_enq2 :',df['first_prod_enq2'].unique())

Marital_status :  ['Married' 'Single']
Education : ['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
Gender : ['M' 'F']
last_prod_enq2 : ['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
first_prod_enq2 : ['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [ ]:

# Ordinal feature -- EDUCATION
# SSC            : 1
# 12TH           : 2
# GRADUATE       : 3
# UNDER GRADUATE : 3
# POST-GRADUATE  : 4
# OTHERS         : 1
# PROFESSIONAL   : 3

# Others has to be verified by the business end user

education_map = {
    'SSC': 1,
    '12TH': 2,
    'GRADUATE': 3,
    'UNDER GRADUATE': 3,
    'POST-GRADUATE': 4,
    'OTHERS': 1,
    'PROFESSIONAL': 3
}

df['EDUCATION'] = df['EDUCATION'].map(education_map)


In [ ]:
df['EDUCATION'].value_counts()
df['EDUCATION'] = df['EDUCATION'].astype(int)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL              42064 non-null  int64  
 13  time_since_recent_payment  42064 non-null  int

In [ ]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'])

In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

In [ ]:
# 1. Random forest
x = df_encoded.drop('Approved_Flag',axis = 1)
y = df_encoded['Approved_Flag']

In [ ]:
x

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,0.000,0.00,0,0.000,0,0,0,4,1,4,...,False,False,True,False,False,False,False,False,True,False
1,0.000,0.00,0,0.000,0,0,0,0,0,1,...,True,False,False,False,False,False,True,False,False,False
2,0.125,0.00,0,0.000,1,0,0,0,2,6,...,True,False,False,False,False,False,False,False,False,True
3,0.000,0.00,0,0.000,0,0,0,0,3,0,...,False,False,False,False,True,False,False,False,False,False
4,0.000,0.00,1,0.167,0,0,0,0,6,0,...,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,0.333,0.00,0,0.000,0,0,0,0,0,3,...,True,False,False,False,False,False,True,False,False,False
42060,0.000,0.25,1,0.250,0,0,0,0,2,2,...,False,False,False,True,False,False,False,False,False,True
42061,0.500,0.50,1,0.500,0,0,0,0,0,2,...,True,False,False,False,False,False,False,False,False,True
42062,0.000,0.00,1,0.500,0,0,0,0,0,2,...,True,False,False,False,False,False,False,False,False,True


In [ ]:
# in random forest classifier, we dont need to encode categorical target , it can handle itself

x_train , x_test , y_train , y_test = train_test_split(x,y,test_size = 0.2 , random_state = 42)

In [ ]:
rf = RandomForestClassifier(n_estimators = 200 , random_state = 42)
rf.fit(x_train , y_train)
y_pred = rf.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)
print(classification_report(y_test,y_pred))

0.7636990372043266
              precision    recall  f1-score   support

          P1       0.84      0.70      0.76      1014
          P2       0.80      0.93      0.86      5045
          P3       0.44      0.21      0.29      1325
          P4       0.72      0.73      0.72      1029

    accuracy                           0.76      8413
   macro avg       0.70      0.64      0.66      8413
weighted avg       0.74      0.76      0.74      8413



In [ ]:
import xgboost as xgb
# we need to convert categorical target into label encoders

from sklearn.preprocessing import LabelEncoder

xgb = xgb.XGBClassifier(objective='multi:softmax', num_class =4)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_train , x_test , y_train , y_test = train_test_split(x,y_encoded,test_size = 0.2 , random_state = 42)

xgb.fit(x_train,y_train)
y_pred = xgb.predict(x_test)

accuracy = accuracy_score(y_test,y_pred)
print(accuracy)
print(classification_report(y_test,y_pred))



0.7783192677998336
              precision    recall  f1-score   support

           0       0.82      0.76      0.79      1014
           1       0.83      0.91      0.87      5045
           2       0.48      0.31      0.37      1325
           3       0.73      0.74      0.73      1029

    accuracy                           0.78      8413
   macro avg       0.71      0.68      0.69      8413
weighted avg       0.76      0.78      0.76      8413



In [ ]:
for i , v in enumerate(label_encoder.classes_):
  print(f"{v} --> {i}")

P1 --> 0
P2 --> 1
P3 --> 2
P4 --> 3


In [ ]:
# Decision Trees Classifier

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth = 20 , min_samples_split = 10)

dt.fit(x_train,y_train)
y_pred = dt.predict(x_test)

accuracy = accuracy_score(y_test,y_pred)
print(accuracy)
print(classification_report(y_test,y_pred))


0.7089028883870201
              precision    recall  f1-score   support

           0       0.72      0.72      0.72      1014
           1       0.81      0.82      0.82      5045
           2       0.34      0.33      0.34      1325
           3       0.65      0.62      0.64      1029

    accuracy                           0.71      8413
   macro avg       0.63      0.62      0.63      8413
weighted avg       0.71      0.71      0.71      8413



In [ ]:
# here xgboost gives the best accuracy among the base models , so we need to do hyperparameter tuning

# trying by applying standard Scaler

import xgboost as xgb

from sklearn.preprocessing import StandardScaler

columns_to_be_scaled = ['Age_Oldest_TL','Age_Newest_TL','time_since_recent_payment',
'max_recent_level_of_deliq','recent_level_of_deliq',
'time_since_recent_enq','NETMONTHLYINCOME','Time_With_Curr_Empr']

df_scaled = df_encoded.copy()

for i in columns_to_be_scaled:
  column_data = df_scaled[i].values.reshape(-1,1)
  scaler = StandardScaler()
  scaled_column = scaler.fit_transform(column_data)
  df_scaled[i] = scaled_column

xgb_classifier = xgb.XGBClassifier(objective = 'multi:softmax',num_class = 4)
x_scaled = df_scaled.drop('Approved_Flag',axis = 1)
y_scaled = y_encoded
# y_encoded = y.LabelEncoder().fit_transform(y) , y = df_encoded['Approved_Flag']

x_train_scaled , x_test_scaled , y_train_scaled , y_test_scaled = train_test_split(x_scaled,y_scaled,test_size = 0.2 , random_state = 42)

xgb_classifier.fit(x_train_scaled , y_train_scaled)
y_pred_scaled = xgb_classifier.predict(x_test_scaled)

accuracy = accuracy_score(y_test_scaled,y_pred_scaled)
print(accuracy)
print(classification_report(y_test_scaled,y_pred_scaled))

0.7783192677998336
              precision    recall  f1-score   support

           0       0.82      0.76      0.79      1014
           1       0.83      0.91      0.87      5045
           2       0.48      0.31      0.37      1325
           3       0.73      0.74      0.73      1029

    accuracy                           0.78      8413
   macro avg       0.71      0.68      0.69      8413
weighted avg       0.76      0.78      0.76      8413



Here , Nearly no improvement in accuracy even after scaling , so now we tune without scaling

In [ ]:

# Hyperparameter tuning in xgboost
from sklearn.model_selection import GridSearchCV
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

# Define the XGBClassifier with the initial set of hyperparameters
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

# Define the parameter grid for hyperparameter tuning

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with the best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(x_test, y_test)
print("Test Accuracy:", accuracy)

# Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}


# Based on risk appetite of the bank, you will suggest P1,P2,P3,P4 to the business end user


Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}
Test Accuracy: 0.7811719957209081


In [ ]:
# hyperparameter tuning with loops

param_grid = {
    'colsample_bytree':[0.1,0.3,0.5,0.7,0.9],
    'learning_rate':[0.001,0.01,0.1,1],
    'max_depth':[3,5,8,10],
    'alpha':[1,10,100],
    'n_estimators':[10,50,100]
}

index = 0

answers_grid = {
    'combination' : [],
    'train_accuracy' : [],
    'test_accuracy' : [],
    'colsample_bytree': [],
    'learning_rate': [],
    'max_depth': [],
    'alpha': [],
    'n_estimators': []
}

for colsample_bytree in param_grid['colsample_bytree']:
  for learning_rate in param_grid['learning_rate']:
    for max_depth in param_grid['max_depth']:
      for alpha in param_grid['alpha']:
        for n_estimators in param_grid['n_estimators']:
          index = index + 1
          xgb_model = xgb.XGBClassifier(colsample_bytree = colsample_bytree,
                                        learning_rate = learning_rate,
                                        max_depth = max_depth,
                                        alpha = alpha,
                                        n_estimators = n_estimators)

          y = df_encoded['Approved_Flag']
          x = df_encoded.drop(['Approved_Flag'],axis = 1)

          y_encoded = LabelEncoder().fit_transform(y)

          x_train , x_test , y_train , y_test = train_test_split(x,y_encoded,test_size = 0.2 , random_state = 42)

          xgb_model.fit(x_train,y_train)

          #predict on training and testing data
          y_pred_train = xgb_model.predict(x_train)
          y_pred_test = xgb_model.predict(x_test)

          train_accuracy = accuracy_score(y_train , y_pred_train)
          test_accuracy = accuracy_score(y_test, y_pred_test)

          # include all params in list

          answers_grid ['combination']   .append(index)
          answers_grid ['train_accuracy']    .append(train_accuracy)
          answers_grid ['test_accuracy']     .append(test_accuracy)
          answers_grid ['colsample_bytree']   .append(colsample_bytree)
          answers_grid ['learning_rate']      .append(learning_rate)
          answers_grid ['max_depth']          .append(max_depth)
          answers_grid ['alpha']              .append(alpha)
          answers_grid ['n_estimators']       .append(n_estimators)

          # Print results for this combination

          print(f"Combination {index}")
          print(f"colsample_bytree: {colsample_bytree}, learning_rate: {learning_rate}, max_depth: {max_depth}, alpha: {alpha}, n_estimators: {n_estimators}")
          print(f"Train Accuracy: {train_accuracy:.2f}")
          print(f"Test Accuracy : {test_accuracy :.2f}")
          print("-" * 30)





Combination 1
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 10
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 2
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 50
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 3
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 100
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 4
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 10
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 5
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 50
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 6
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 100
Train Accuracy: 0.61
Test Accu

In [ ]:
answers_df = pd.DataFrame(answers_grid)

In [ ]:
print(answers_df.sort_values(by='test_accuracy', ascending=False).head(5))
